In [20]:
import minsearch
import json
from openai import OpenAI
from dotenv import dotenv_values

In [21]:
API_KEY = dotenv_values(".env")["API_KEY"]

In [3]:
with open("documents.json", "rt") as f_in:
    docs_raw = json.load(f_in)

In [5]:
documents = []
for course_dict in docs_raw:
    for doc in course_dict["documents"]:
        doc["course"] = course_dict["course"]
        documents.append(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [8]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
    )

In [41]:
q = "The course has already started, can i enroll now ?"

In [12]:
index.fit(documents)

In [17]:
boost = {"question": 3.0, "section": 0.5, }
results = index.search(
    query = q,
    filter_dict={"course": "data-engineering-zoomcamp"},
    boost_dict= boost, 
    num_results=5,
    
)

In [18]:
results

[{'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question':

In [39]:
client = OpenAI(api_key=API_KEY, base_url="https://api.perplexity.ai")

In [26]:
response = client.chat.completions.create(
    model = "llama-3-sonar-small-32k-chat",
    messages = [{
        "role": "user",
        "content": q
    }]
)

In [29]:
response.choices[0].message.content

"I'm happy to help However, I have to clarify that I'm a large language model, I don't have access to specific information about your course or enrollment status. But I can try to provide some general guidance.\n\nIf the course has already started, it's unlikely that you'll be able to enroll now. Most courses have a specific start and end date, and once the course has begun, it's usually not possible to join mid-stream.\n\nHowever, it's always worth checking with the course instructor or administrator to see if there are any exceptions or special arrangements that can be made. They may be able to provide more information on whether it's possible to enroll late or if there are any alternative options available.\n\nAdditionally, you can also reach out to the course provider or institution to inquire about their policies on late enrollment or course extensions. They may have specific procedures in place for handling late enrollments or exceptions.\n\nRemember to always check the course de

In [30]:
propmt_template = """"You're a course teaching assistant. Answer the QUESTION based on CONTEXT. 
Use only the facts from CONTEXT when answering the QUESTION.
If the CONTEXT doesnt contain the answer, output NONE.

QUESTION: {question}

CONTEXT: {context}""" ## specify the role # prompt engineering 

In [32]:
context = ""

for doc in results:
    context = context + f"section : {doc['section']} \nquestion: {doc['question']} \n answer: {doc['text']}\n\n"

In [42]:
prompt = propmt_template.format(question=q, context = context).strip()

In [43]:
print(prompt)

"You're a course teaching assistant. Answer the QUESTION based on CONTEXT. 
Use only the facts from CONTEXT when answering the QUESTION.
If the CONTEXT doesnt contain the answer, output NONE.

QUESTION: The course has already started, can i enroll now ?

CONTEXT: section : General course-related questions 
question: Course - Can I follow the course after it finishes? 
 answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section : General course-related questions 
question: Course - When will the course start? 
 answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1
Subscribe to course pu

In [47]:
# putting this to gpt:
response = client.chat.completions.create(
    model = "llama-3-sonar-large-32k-chat",
    messages = [{"role": 'user', "content": prompt}]
)

response.choices[0].message.content

'Yes.'